In [2]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 상대경로: 현재 작업 디렉토리 기준으로 폴더 생성
base_save_dir = os.path.join(".", "crawwing")
artist_folder_name = "오승윤"
artist_folder_path = os.path.join(base_save_dir, artist_folder_name)
os.makedirs(artist_folder_path, exist_ok=True)  # 폴더가 없으면 자동 생성

# 영어 이름으로 저장할 파일명
csv_filename = "Oh_SyngYoon.csv"
xlsx_filename = "Oh_SyngYoon.xlsx"
csv_path = os.path.join(artist_folder_path, csv_filename)
xlsx_path = os.path.join(artist_folder_path, xlsx_filename)

headers = {
    "User-Agent": "Mozilla/5.0"
}

def safe_filename(s):
    return "".join(c if c.isalnum() or c in "._-" else "_" for c in s)

all_data = []

for page_num in range(1, 9):
    PAGE_URL = f"http://kartprice.net/price/list/?search_val=%EC%98%A4%EC%8A%B9%EC%9C%A4&type=works&sort=trade&sorttype=desc&page={page_num}"
    print(f"크롤링 중: 페이지 {page_num}")
    response = requests.get(PAGE_URL, headers=headers, timeout=10)
    soup = BeautifulSoup(response.text, "html.parser")
    rows = soup.select("table.table_newStyle1 tr[onclick]")
    print(f"페이지 {page_num}에서 {len(rows)}개 작품 발견")

    for idx, row in enumerate(rows, 1):
        # 이미지
        img_tag = row.select_one("td.pic img")
        img_url = img_tag["src"].strip() if img_tag else ""
        img_ext = os.path.splitext(img_url)[-1] if img_url else ".jpg"
        
        # 작품명
        title = row.select_one("td.title p")
        title = title.get_text(strip=True) if title else ""

        # 작가명
        artist = row.select_one("td.name p")
        artist = artist.get_text(strip=True) if artist else ""

        # 제작년도/규격
        year = ""
        size = ""
        year_size_td = row.select("td")[4] if len(row.select("td")) > 4 else None
        if year_size_td:
            ps = year_size_td.find_all("p")
            year = ps[0].get_text(strip=True) if len(ps) > 0 else ""
            size = ps[1].get_text(strip=True) if len(ps) > 1 else ""

        # 장르/기법
        genre = ""
        technique = ""
        genre_td = row.select("td")[5] if len(row.select("td")) > 5 else None
        if genre_td:
            ps = genre_td.find_all("p")
            genre = ps[0].get_text(strip=True) if len(ps) > 0 else ""
            technique = ps[1].get_text(strip=True) if len(ps) > 1 else ""

        # 출품처/거래일
        auction_house = ""
        auction_date = ""
        auction_td = row.select("td")[6] if len(row.select("td")) > 6 else None
        if auction_td:
            ps = auction_td.find_all("p")
            auction_house = ps[0].get_text(strip=True) if len(ps) > 0 else ""
            auction_date = ps[1].get_text(strip=True) if len(ps) > 1 else ""

        # 낙찰가격
        price_td = row.select_one("td.money")
        price = price_td.get_text(strip=True) if price_td else ""

        # 이미지 저장 파일명 생성
        img_filename = f"{safe_filename(artist)}_{safe_filename(title)}_{page_num}_{idx}{img_ext}"
        img_path = os.path.join(artist_folder_path, img_filename)

        # 이미지 다운로드 (이미지 저장만, 데이터에는 경로/URL 저장X)
        if img_url:
            try:
                img_resp = requests.get(img_url, headers=headers, timeout=10)
                if img_resp.status_code == 200:
                    with open(img_path, "wb") as f:
                        f.write(img_resp.content)
            except Exception as e:
                print(f"이미지 다운로드 실패: {img_url} ({e})")

        # 데이터 저장 (이미지경로, 이미지URL 제외)
        all_data.append({
            "작가명": artist,
            "작품명": title,
            "제작년도": year,
            "규격": size,
            "장르": genre,
            "기법": technique,
            "경매사": auction_house,
            "거래일": auction_date,
            "낙찰가격": price
        })

        print(f"페이지 {page_num} - {idx}/{len(rows)} 작품 저장 완료: {artist} - {title}")

# 크롤링 완료 후 CSV, XLSX 저장
df = pd.DataFrame(all_data)
df.to_csv(csv_path, index=False, encoding="utf-8-sig")
df.to_excel(xlsx_path, index=False)

print(f"\n크롤링 완료! 총 {len(df)}건 저장")
print(f"CSV 저장 위치: {csv_path}")
print(f"엑셀 저장 위치: {xlsx_path}")

# 결과 일부 출력
print(df.head())


크롤링 중: 페이지 1
페이지 1에서 10개 작품 발견
페이지 1 - 1/10 작품 저장 완료: 오승윤 - 무제
페이지 1 - 2/10 작품 저장 완료: 오승윤 - 해녀도
페이지 1 - 3/10 작품 저장 완료: 오승윤 - 무제
페이지 1 - 4/10 작품 저장 완료: 오승윤 - 호수
페이지 1 - 5/10 작품 저장 완료: 오승윤 - 물고기
페이지 1 - 6/10 작품 저장 완료: 오승윤 - 무제
페이지 1 - 7/10 작품 저장 완료: 오승윤 - 청산
페이지 1 - 8/10 작품 저장 완료: 오승윤 - 산
페이지 1 - 9/10 작품 저장 완료: 오승윤 - 무제
페이지 1 - 10/10 작품 저장 완료: 오승윤 - 풍수
크롤링 중: 페이지 2
페이지 2에서 10개 작품 발견
페이지 2 - 1/10 작품 저장 완료: 오승윤 - 무제
페이지 2 - 2/10 작품 저장 완료: 오승윤 - 무제
페이지 2 - 3/10 작품 저장 완료: 오승윤 - 무제
페이지 2 - 4/10 작품 저장 완료: 오승윤 - 누드
페이지 2 - 5/10 작품 저장 완료: 오승윤 - Autumn
페이지 2 - 6/10 작품 저장 완료: 오승윤 - 가을
페이지 2 - 7/10 작품 저장 완료: 오승윤 - 법성포마을
페이지 2 - 8/10 작품 저장 완료: 오승윤 - 무제
페이지 2 - 9/10 작품 저장 완료: 오승윤 - 여인 누드
페이지 2 - 10/10 작품 저장 완료: 오승윤 - 무제
크롤링 중: 페이지 3
페이지 3에서 10개 작품 발견
페이지 3 - 1/10 작품 저장 완료: 오승윤 - 무제
페이지 3 - 2/10 작품 저장 완료: 오승윤 - 도강
페이지 3 - 3/10 작품 저장 완료: 오승윤 - 무제
페이지 3 - 4/10 작품 저장 완료: 오승윤 - 무제
페이지 3 - 5/10 작품 저장 완료: 오승윤 - 가을
페이지 3 - 6/10 작품 저장 완료: 오승윤 - 무제
페이지 3 - 7/10 작품 저장 완료: 오승윤 - 누드
페이지 3 - 8/10 작품 저장 완료: 오승윤 - 다